In [1]:
import pandas as pd
import dill

from multiprocessing import shared_memory
from fb_shared_memory import FbSharedMemory

In [7]:
df = pd.read_csv("climatechange_tweets_all.csv")
df.head()

,Unnamed: 0,tweet_id,tweet_created_at,language,user_screen_name,user_created_at,user_id,followers_count,friends_count,time_zone,utc_offset,retweeted_status,retweet_id,retweet_user_screen_name,retweet_user_id
0,0,1068223635024941056,2018-11-29 19:22:15,en,JGuerreroN13,2017-06-07 20:21:48,872549171680694272,245,419,NaN,NaN,1,1.068067e+18,UNFCCC,1.746392e+07
1,1,1068223643765940226,2018-11-29 19:22:17,en,t_rustyboy1720,2017-09-23 03:53:34,911438362271031296,63,232,NaN,NaN,1,1.068211e+18,saskboy,6.634632e+06
2,2,1068223645330432001,2018-11-29 19:22:18,en,MissCJWright,2009-03-15 16:24:32,24544855,1383,4585,NaN,NaN,1,1.068223e+18,EcoSenseNow,1.392956e+09
3,3,1068223649222664194,2018-11-29 19:22:19,en,al_f,2009-07-17 00:05:50,57490401,333,377,NaN,NaN,1,1.068182e+18,ClimateComms,3.626422e+08
4,4,1068223660299677696,2018-11-29 19:22:21,en,Hawknana_,2013-12-21 03:16:04,2255917724,1751,3409,NaN,NaN,1,1.068187e+18,PortlandBEM,1.803082e+07


In [8]:
%%time
# Serializing the dataframe with dill, then adding it to shared memory
df_bytestring = dill.dumps(df)
shm_a = shared_memory.SharedMemory(create=True, name = "dill_shm", size=len(df_bytestring) + 1000)
shm_a.buf[:len(df_bytestring)] = df_bytestring
print(len(df_bytestring))

63245615
CPU times: user 6.62 s, sys: 103 ms, total: 6.72 s
Wall time: 6.8 s


In [9]:
%%time
# Serializing the dataframe with flatbuffers, then adding it to shared memory
shm = FbSharedMemory()
shm.add_dataframe("df1", df)

CPU times: user 1.25 ms, sys: 726 µs, total: 1.97 ms
Wall time: 1.58 ms
